In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# CSV file into a DataFrame
df = pd.read_csv('./csv_data/new_transactions.csv', parse_dates=['transaction_create_date'])

# Currency-to-Country Dictionary/Mapping
currency_to_country = {
    'HUF': 'Hungary',
    'EUR': 'European Union',
    'GBP': 'United Kingdom',
    'USD': 'United States'}

# Adding 'country' column based on the mapping
df['country'] = df['transaction_currency'].map(currency_to_country)

# Adding the currency prefix to the merchant name column
df['merchant_id'] = df['transaction_currency'] + '_' + df['merchant_id'].astype(str)

# Total sum of the transaction_total_amount column
total_tpv = df['transaction_total_amount'].sum()
print("Total TPV (Transaction Value):", total_tpv)

# Changing DataFrame to CSV with added prefixes
df.to_csv('prefixed_transactions.csv', index=False)

# TPV over time chart us below:

# Group by Country and Week, summing up transaction_total_amount
df['week'] = df['transaction_create_date'].dt.to_period('W').astype(str)
grouped_df = df.groupby(['country', 'week']).agg({'transaction_total_amount': 'sum'}).reset_index()

non_hun_data = grouped_df[grouped_df['country'] != 'Hungary']

# Separate Hungary data
hun_data = grouped_df[grouped_df['country'] == 'Hungary']

# Plotting
fig, ax1 = plt.subplots(figsize=(12, 6))

# Plotting left y-axis for other countries
for country, data in non_hun_data.groupby('country'):
    ax1.plot(data['week'], data['transaction_total_amount'], label=country)

ax1.set_title('Total Transaction Value (TPV) for Each Country Over Weeks')
ax1.set_xlabel('Week')
ax1.set_ylabel('Total Transaction Value (TPV) - Other Countries')
ax1.legend(loc='upper left')

# Customize x-axis ticks and labels
xticks = grouped_df['week'].unique()[::4]
ax1.set_xticks(xticks)
ax1.tick_params(axis='x', rotation=45)

# Creating y-axis for Hungary
ax2 = ax1.twinx()
ax2.plot(hun_data['week'], hun_data['transaction_total_amount'], label='Hungary', color='red')
ax2.set_ylabel('Total Transaction Value (TPV) - Hungary')
ax2.legend(loc='upper right')

# Switch two y lines from scientific notation to normal numbers
ax1.ticklabel_format(style='plain', axis='y')
ax2.ticklabel_format(style='plain', axis='y')

# Set y-axis limits for both axes
ax1.set_ylim(0, max(non_hun_data['transaction_total_amount']) + 1000000)
ax2.set_ylim(0, max(hun_data['transaction_total_amount']) + 10000000000)
ax1.set_xlim(grouped_df['week'].min(), grouped_df['week'].max())

plt.tight_layout()  # Adjust layout for better appearance
plt.show()


: 